# Matrix product state representation for ground states

$
\require{physics}
\def\bm{\boldsymbol}
$

## Introduction

Clearly, God did not conceive the universe as a point because God would not be
able to take the outer product of two points to obtain vectors.
So in the beginning, there were at least two vectors, and God took the outer
product to make higher order tensors, the inner product to make lower order 
tensors: these were the binary operators on tensors.
God defined these tensors over a field and also used the transpose and inverse.
We believe God may have saved us because God invented index notation 
so that any of this could make sense.
Therefore tensors, their fields, their classes, their algebras, and their calculus
is the only way that it is convenient to express the geometry of the universe.

God's intent for the universe can at least be expressed by the following
statement: 
$$
\text{Let there be $\left(\prod_n F \right)^U$ for some field $F$ and positive integers $n, U$.}
$$
As explained above, it would have sufficed to have $n>1$ and $U$ equal unity (1)
in order for God to obtain a vector field to create the universe, which explains
the notation.

Sometimes smart people find ways to represent large tensors by smaller tensors,
and this assignment is originated in this observation.
The goal is to rewrite a tensor of dimension $d^L$ which describes the state
of a qu$d$it chain of known $L$ength as a matrix product.
We do so using the SVD and Schmidt decomposition techniques explored earlier.

Other projects seem pretty useful to look into for helpful
[resources](https://tensornetwork.org/) and
[software](https://github.com/google/TensorNetwork).
This is a [useful paper](https://arxiv.org/abs/1008.3477) cited in the
assignment on the subject.

## MPS representation

One asks how you actually do this task efficiently, which comes down to knowing
how to represent the MPS wavefunction in memory.
Choose a positive integer value for $d$ and $L$.
The representation is of the form
$$
    \psi_{\sigma_1, \dots, \sigma_L} = 
        \sum_{\alpha_0, \dots, \alpha_L}
        \left( \prod_{i=0}^{L-1} A_{\alpha_i \alpha_{i+1}}^{\sigma_{i+1}} \right)
$$
where
\begin{align}
    \dim(\alpha_i) \leq 
        \begin{cases}
            d^i
                &\qq{i \leq L/2}
            \\\\
            d^{L-i}
                &\qq{i > L/2}
        \end{cases}
    .
\end{align}
The form of this dimensionality is due to the way the mps representation is 
constructed: a tensor is reshaped and has its SVD taken many times,
each new time after another reshaping and SVD.
Essentially these dimensions are the maximal ranks of a matrix as it is reshaped
from $(1, d^L)$ to $(d^L, 1)$ when exchanging rows to columns $d$ at a time.
Now an approximation scheme for MPS is a map:
\begin{align}
    r(i) : \{i\} \to \{1, \dots, \max(1, \min(d r(i-1), \dim(\alpha_i))\}
\end{align}
with $r(0) = 1$
that specifies the number of rows retained at each bond index.
This is a finite but large function space, and of the many approximation schemes
it often makes sense to choose a simple one, such as
$r(i) = \min(\chi, \dim(\alpha_i))$ for constant $\chi$.

### Scaling

The scaling of storage requirements as a function of the approximation scheme
can be calculated succinctly as $\sum_i r(i) r(i+1)$.
Taking $r=\dim(\alpha_i)$ makes for an inefficient full representation of the
because of the $d^{L+1} > d^L$ scaling.
For what truncation ranks $r$ is MPS an efficient storage scheme?
We will test the accuracy of the scheme later.

## Program

- $d=2, r=\chi=1$ ($r=1$ is a special case where the storage format can be optimized)
- For $h \in \{ 1, 5/4 \}$
- Take the ground state in the open system at large L
- Compute the MPS approximation of the wavefunction, varying the bond length $k$
- Calculate the actual reduction in storage space (np.size)
- Contract the indices of the tensors of MPS to obtain exponentially large
$\ket{\tilde{\psi}_{gs} (k)}$.
- Compute the overlap $\braket{\tilde{\psi}\_{gs} (k)}{\psi\_{gs}}$

### Efficient calculations with MPS

- Use MPS to calculate
$E(k) = \ev{H}{\tilde{\psi}\_{gs} (k)} / \braket{\tilde{\psi}\_{gs} (k)}$
- Compute the same correlation functions as in Assignment 1 at both values of 
the order parameter and study the convergence in $k$.

In [ ]:
import numpy as np
import scipy.sparse.linalg as sla

from ph121c_lxvm import basis, tfim, tests

In [ ]:
for oper_params in tests.tfim.param_sweep(
    L = [6],
    h = [0.3],
    bc= ['c'],
):
    job = dict(
        oper=tfim.z.H_sparse,
        oper_params=oper_params,
        solver=sla.eigsh,
        solver_params={ 
            'k' : 6, 
            'which' : 'BE',
        },
    )
    evals, evecs = tfim.data.obtain(**job)

In [ ]:
def rank (i, L, d=2):
    chi = 2
    return max(1, min(chi, basis.mps.dim_mps(i, L, d)))

In [ ]:
A = basis.mps.my_mps(evecs[:, 0], basis.mps.dim_mps, 6, 2)

In [ ]:
np.linalg.norm(A.v)

In [ ]:
A.contract_bonds() / np.linalg.norm(A.contract_bonds())

## Results

## Discussion